In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from qgsw.utils.sorting import sort_files

from qgsw.run_summary import RunSummary
import torch
import matplotlib.pyplot as plt

ROOT = Path(os.path.abspath('')).parent

# Profiles

In [ ]:
plots = [
# {
# "field": "pv",
# "layer": 0,
# "input": "one_layer_baroclinic_30km",
# "output": "1L_baroclinic_30km",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# },
{
"field": "pv",
"layer": 0,
"input": "two_layers_baroclinic_30km",
"output": "two_layers_baroclinic_30km_",
"steps": [0, 33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 1,
"input": "two_layers_baroclinic_30km",
"output": "two_layers_baroclinic_30km__",
"steps": [0, 33084, 62492, 91898],
"colorscale": "RdBu_r",
},
# {
# "field": "pv",
# "layer": 0,
# "input": "two_layers_baroclinic_30km",
# "output": "2L_baroclinic_30km",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# },
# {
# "field": "pv",
# "layer": 1,
# "input": "two_layers_baroclinic_30km",
# "output": "2L_baroclinic_30km_bottom",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# },
# {
# "field": "pv",
# "layer": 0,
# "input": "one_layer_baroclinic_100km",
# "output": "1L_baroclinic_100km",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# },
# {
# "field": "pv",
# "layer": 0,
# "input": "two_layers_baroclinic_100km",
# "output": "2L_baroclinic_100km",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# },
# {
# "field": "pv",
# "layer": 1,
# "input": "two_layers_baroclinic_100km",
# "output": "2L_baroclinic_100km_bottom",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# },
# {
# "field": "pv",
# "layer": 0,
# "input": "one_layer_barotropic_100km",
# "output": "1L_barotropic_100km",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# },
# {
# "field": "pv",
# "layer": 0,
# "input": "two_layers_barotropic_100km",
# "output": "2L_barotropic_100km",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",},
# {
# "field": "pv",
# "layer": 1,
# "input": "two_layers_barotropic_100km",
# "output": "2L_barotropic_100km_bottom",
# "steps": [33084, 62492, 91898],
# "colorscale": "RdBu_r",
# }
]

In [ ]:
import os
from pathlib import Path

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from qgsw.run_summary import RunSummary
import toml

ROOT = Path(os.path.abspath('')).parent


plots_config = toml.load(ROOT.joinpath("config/save_plots.toml"))

for plot_config in plots:
    field = plot_config["field"]
    layer = plot_config["layer"]

    input_folder = ROOT.joinpath(f"output/g5k/{plot_config['input']}")
    output_folder = ROOT.joinpath(f"output/snapshots/{plot_config['output']}")
    if not output_folder.is_dir():
        output_folder.mkdir(parents=True)

    summary = RunSummary.from_file(input_folder.joinpath("_summary.toml"))
    config = summary.configuration
    x_min, x_max = config.space.box.x_min, config.space.box.x_max
    y_min, y_max = config.space.box.y_min, config.space.box.y_max
    
    offset = 24

    datas = []

    for i,step in enumerate(plot_config["steps"]):
        file = input_folder.joinpath(f"{config.model.prefix}{step}.npz")

        data = np.load(file)[field][0, layer, ...][offset:-offset,offset:-offset]

        datas.append(data)

    zmax = max(np.max(np.abs(data)) for data in datas)
    zmin = -zmax

    for i,data in enumerate(datas):
        colorbar = go.heatmap.ColorBar(
            exponentformat="e",
            showexponent="all",
            title={"text": "Potential Vorticity (s⁻¹)", "side": "right"},
            thickness=100
        )

        x = np.linspace(x_min / 1000, x_max / 1000, config.space.nx)[offset:-offset]
        y = np.linspace(y_min / 1000, y_max / 1000, config.space.ny)[offset:-offset]

        heatmap = go.Heatmap(
            z=data.T,
            x=x,
            y=y,
            colorscale=px.colors.diverging.RdBu_r,
            zmin=zmin,
            zmax=zmax,
            colorbar=colorbar,
            showscale=i == 2,
        )
        fig = go.Figure()
        fig.add_trace(heatmap)#, row=1, col=i+1)
        fig.add_shape(type="rect", xref=f"x", yref=f"y", x0=x[0], x1=x[-1], y0=y[0], y1=y[-1], line=dict(color="black", width=2))

        fig.update_layout(
            # template="simple_white",
            autosize=True,
            margin=dict(l=20, r=20, t=20, b=20),
            width=1000+ (400 * (i==2)) + 100 * (i==0),
            height=1000 ,
            font={"size": 60, "color":"black"},
            xaxis={"scaleanchor": "y", "constrain": "domain"},
            yaxis={"scaleanchor": "x", "constrain": "domain"},
        )

        fig.update_xaxes(
            title={"text": "X (km)"},
            exponentformat="none",
            dtick = (x_max - x_min) / 1000 / 6 - ((x_max - x_min)/ 1000 / 6 )% 50,
            tick0 = 0,
        )

        fig.update_yaxes(
            title={"text": "Y (km)"},
            exponentformat="none",
            dtick = (y_max - y_min) / 1000 / 6 - ((y_max - y_min)/1000/6) % 50,
            tick0 = 0,
            ticksuffix = "  ",
            visible = i == 0,
        )
        fig.show()
        # fig.write_image(output_folder.joinpath(f"snapshot_{plot_config['steps'][i]}.png"))

# RMSE

## Reference RMSES

In [ ]:
plots_compare = [
[
    {
    "field": "pv",
    "layer": 0,
    "name": "One Layer vs Two Layers",
    "input": ["one_layer_baroclinic_30km", "two_layers_baroclinic_30km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "Modified One Layer vs Two Layers",
    "input": ["pv_baroclinic", "two_layers_baroclinic_30km"],
    },
],
[
    {
    "field": "pv",
    "layer": 0,
    "name": "One Layer vs Two Layers",
    "input": ["one_layer_barotropic_100km", "two_layers_barotropic_100km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "Modified One Layer vs Two Layers",
    "input": ["pv_barotropic", "two_layers_barotropic_100km"],
    },
],
]

In [ ]:
def loss(x:np.ndarray, y:np.ndarray) -> float:
    return np.sqrt(np.mean(np.square(x - y))) / 9.375e-5

In [ ]:
import os
from pathlib import Path

import numpy as np
import plotly.graph_objects as go
from qgsw.run_summary import RunSummary
import toml
from qgsw.utils.sorting import sort_files

ROOT = Path(os.path.abspath('')).parent

for i,plot in enumerate(plots_compare):
    fig = go.Figure()

    for j, plot_config in enumerate(plot):

        field = plot_config["field"]
        layer = plot_config["layer"]

        input_folder1 = ROOT.joinpath(f"output/g5k/{plot_config['input'][0]}")
        input_folder2 = ROOT.joinpath(f"output/g5k/{plot_config['input'][1]}")

        summary1 = RunSummary.from_file(input_folder1.joinpath("_summary.toml"))
        config1 = summary1.configuration
        summary2 = RunSummary.from_file(input_folder2.joinpath("_summary.toml"))
        config2 = summary2.configuration

        steps_1, files_1 = sort_files(list(input_folder1.glob(f"{config1.model.prefix}*.npz")),config1.model.prefix,".npz")
        steps_2, files_2 = sort_files(list(input_folder2.glob(f"{config2.model.prefix}*.npz")),config2.model.prefix,".npz")

        x_min, x_max = config1.space.box.x_min, config1.space.box.x_max
        y_min, y_max = config1.space.box.y_min, config1.space.box.y_max
        
        offset = 0
        losses = []
        times = []

        for k,file1 in enumerate(files_1):

            file2 = files_2[k]

            data1 = np.load(file1)[field][0, layer, ...]
            data2 = np.load(file2)[field][0, layer, ...]

            if offset != 0 :
                data1 = data1[offset:-offset,offset:-offset]
                data2 = data2[offset:-offset,offset:-offset]

            losses.append(loss(data1, data2))
            times.append(steps_1[k] * config1.simulation.dt / 3600 / 24)
        
        scatter = go.Scatter(
            x=times, 
            y=losses,
            name = plot_config["name"],
            mode = "lines",
            line= dict(width=8),
        )

        fig.add_trace(scatter)

    fig.update_layout(
        template="plotly_white",
        autosize=True,
        width=1500 + 250 * (i==0),
        height=1200 ,
        font={"size": 60, "color":"black"},
    )
    fig.update_layout(
        margin=dict(l=20 + 100 * (i==0), r=20, t=20, b=20),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
        )
    )
    fig.update_xaxes(
        title={"text": "Time (day)"},
        exponentformat="e",
        mirror=True,
        linewidth=3,
        showgrid=False,
        linecolor="black",
    )

    fig.update_yaxes(
        title={"text": f"{'RMSE (normalized)' if i==0 else ''}"},
        range=[-0.0001, 0.0075],
        exponentformat="none",
        ticksuffix = "  ",
        tickprefix = "  ",
        tick0 = 0,
        dtick = 0.0025,
        mirror=True,
        showgrid=True,
        gridwidth=2,
        linewidth=3,
        showticklabels=i==0,
        linecolor="black",

    )
    fig.show()

## Experiments RMSES

In [ ]:
plot = [
    # {
    # "field": "pv",
    # "layer": 0,
    # "name": "α = 0 vs 2L",
    # "input": ["sf_alpha_0", "two_layers_baroclinic_100km"],
    # },
    # {
    # "field": "pv",
    # "layer": 0,
    # "name": "α = 0.1 vs 2L",
    # "input": ["sf_alpha_0_1", "two_layers_baroclinic_100km"],
    # },
    # {
    # "field": "pv",
    # "layer": 0,
    # "name": "α = 0.5 vs 2L",
    # "input": ["sf_alpha_0_5", "two_layers_baroclinic_100km"],
    # },
    {
    "field": "pv",
    "layer": 0,
    "name": "1L vs 2L",
    "input": ["one_layer_baroclinic_100km", "two_layers_baroclinic_100km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "α = 0.25 vs 2L",
    "input": ["sf_alpha_0_25", "two_layers_baroclinic_100km"],
    },
    # {
    # "field": "pv",
    # "layer": 0,
    # "name": "SF Sigmoid 0.2 vs 2L",
    # "input": ["sf_changing_sigmoid_0_2", "two_layers_baroclinic_100km"],
    # },
    # {
    # "field": "pv",
    # "layer": 0,
    # "name": "SF 0.2 vs 2L",
    # "input": ["sf_alpha_0_2", "two_layers_baroclinic_100km"],
    # },
    {
    "field": "pv",
    "layer": 0,
    "name": "θ = 0.25 vs 2L",
    "input": ["sf_changing_0_25", "two_layers_baroclinic_100km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "θ = 0.5 vs 2L",
    "input": ["sf_changing_0_5", "two_layers_baroclinic_100km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "θ = 0.75 vs 2L",
    "input": ["sf_changing_0_75", "two_layers_baroclinic_100km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "θ = 1 vs 2L",
    "input": ["sf_changing_1", "two_layers_baroclinic_100km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "θ = 1.25 vs 2L",
    "input": ["sf_changing_1_25", "two_layers_baroclinic_100km"],
    },
    {
    "field": "pv",
    "layer": 0,
    "name": "θ = 1.5 vs 2L",
    "input": ["sf_changing_1_5", "two_layers_baroclinic_100km"],
    },
]

In [ ]:
def loss(x:np.ndarray, y:np.ndarray) -> float:
    return np.sqrt(np.mean(np.square(x - y))) / 9.375e-5

In [ ]:
import os
from pathlib import Path

import numpy as np
import plotly.graph_objects as go
import plotly.colors as pcol
from qgsw.run_summary import RunSummary
import toml
from qgsw.utils.sorting import sort_files

ROOT = Path(os.path.abspath('')).parent

colors =  ["#EF553B","#FECB52"]
colors += pcol.sequential.Viridis[:6]#[:1]

fig = go.Figure()

for j, plot_config in enumerate(plot):

    field = plot_config["field"]
    layer = plot_config["layer"]

    input_folder1 = ROOT.joinpath(f"output/g5k/{plot_config['input'][0]}")
    input_folder2 = ROOT.joinpath(f"output/g5k/{plot_config['input'][1]}")

    summary1 = RunSummary.from_file(input_folder1.joinpath("_summary.toml"))
    config1 = summary1.configuration
    summary2 = RunSummary.from_file(input_folder2.joinpath("_summary.toml"))
    config2 = summary2.configuration

    steps_1, files_1 = sort_files(list(input_folder1.glob(f"{config1.model.prefix}*.npz")),config1.model.prefix,".npz")
    steps_2, files_2 = sort_files(list(input_folder2.glob(f"{config2.model.prefix}*.npz")),config2.model.prefix,".npz")

    x_min, x_max = config1.space.box.x_min, config1.space.box.x_max
    y_min, y_max = config1.space.box.y_min, config1.space.box.y_max
    
    offset = 0
    losses = []
    times = []

    for k,file1 in enumerate(files_1):

        file2 = files_2[k]

        data1 = np.load(file1)[field][0, layer, ...]
        data2 = np.load(file2)[field][0, layer, ...]

        if offset != 0 :
            data1 = data1[offset:-offset,offset:-offset]
            data2 = data2[offset:-offset,offset:-offset]

        losses.append(loss(data1, data2))
        times.append(steps_1[k] * config1.simulation.dt / 3600 / 24)
    
    line = dict(
            width=8,
            color=colors[j],
        )
    if j==0 or j==1:
        line["dash"] = "dashdot"

    scatter = go.Scatter(
        x=times, 
        y=losses,
        name = plot_config["name"],
        mode = f"lines",
        line= line,
    )

    fig.add_trace(scatter)

fig.update_layout(
    template="plotly_white",
    autosize=True,
    width=2500,
    height=1200 ,
    font={"size": 50, "color":"black"},
)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_xaxes(
    title={"text": "Time (day)"},
    exponentformat="e",
    mirror=True,
    linewidth=3,
    showgrid=False,
    linecolor="black",
)

fig.update_yaxes(
    title={"text": "RMSE (normalized)"},
    exponentformat="none",
    ticksuffix = "  ",
    tickprefix = "  ",
    tick0 = 0,
    mirror=True,
    showgrid=True,
    gridwidth=2,
    linewidth=3,
    linecolor="black",

)
fig.show()

# Modes

In [ ]:
import torch


baroclinic_30 = "two_layers_baroclinic_30km"
barotropic_100 = "two_layers_barotropic_100km"
baroclinic_100 = "two_layers_baroclinic_100km"
barotropic_30 = "barotropic_30km_2l"



path = ROOT.joinpath(f"output/g5k/{baroclinic_30}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_baroclinic_30 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

path = ROOT.joinpath(f"output/g5k/{barotropic_100}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_barotropic_100 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

path = ROOT.joinpath(f"output/g5k/{baroclinic_100}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_baroclinic_100 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

path = ROOT.joinpath(f"output/g5k/{barotropic_30}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_barotropic_30 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

all_files = [
    files_baroclinic_30,
    files_barotropic_100,
    files_barotropic_30,
    files_baroclinic_100,
][:]
names = [
    "baroclinic_30km", 
    "barotropic_100km", 
    "barotropic_30km",
    "baroclinic_100km",
]

H1,H2 = run.configuration.model.h
g1, g2 = run.configuration.model.g_prime

A = torch.tensor(
    [
        [1/H1/g1+1/H1/g2, -1/H1/g2],
        [-1/H2/g2, 1/H2/g2]
    ],
    dtype=torch.float64
)

P = torch.linalg.eig(A)[1].real

offset = 24



for j,files in enumerate(all_files):
    modes_1 = []
    modes_2 = []
    times=[]
    for i in range(0,len(steps),1):

        psi = torch.tensor(np.load(files[i])["p"], dtype=torch.float64)[...,offset:-offset,offset:-offset]
        modes = torch.einsum("lm,...mxy->...lxy", torch.inverse(P), psi)
        top = modes[0,0,...]
        bottom = modes[0,1,...]

        mode_1 = torch.mean(torch.square(top))
        mode_2 = torch.mean(torch.square(bottom))

        modes_1.append(mode_1 / (mode_1 + mode_2))
        modes_2.append(mode_2 / (mode_1 + mode_2))

        times.append(steps[i] * run.configuration.simulation.dt / 3600 / 24)

    fig = go.Figure()
    fig.update_layout(
        template="plotly_white",
        autosize=False,
        margin=dict(l=250 * (j==0) + 20, r=20, t=20, b=20),
        width=1000+ (525 * (j==3)) + 200 * (j==0),
        height=1000 ,
        font={"size": 60, "color":"black"},
        # legend=dict(orientation="h", x=0.1, y=0.9)
    )
    fig.update_yaxes(
        range=[-0.1,1.1],
        title = {"text": "Mode Intensity"},
        ticksuffix = "  ",
        tick0=0,
        dtick=0.25,
        visible = j == 0,
    )
    fig.update_xaxes(
        range=[times[0], times[-1]],
        title = {"text": "Time (day)"}
    )
    fig.add_shape(
        type="rect", xref=f"x", yref=f"y", x0=times[0], x1=times[-1], y0=-0.1, y1=1.1, line=dict(color="black", width=3)
    )
    fig.add_trace(
        go.Scatter(
            name = "Baroclinic Mode",
            x = times,
            y = modes_1,
            mode = "lines",
            line= dict(width=10),
            showlegend=(j==3)
        ),
    )
    fig.add_trace(
        go.Scatter(
            name="Barotropic Mode",
            x = times,
            y = modes_2,
            mode = "lines",
            line= dict(width=10),
            showlegend=(j==3)
        ),
    )
    fig.show()
    # fig.write_image(f"../output/snapshots/modes/{names[j]}.png")

# SubLayer Ratios

In [ ]:
# %matplotlib tk

from qgsw.spatial.core.grid_conversion import cell_corners_to_cell_center
import plotly.colors as pcol

folder = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km_long")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

colors =  pcol.sequential.Viridis[::][:6]

fig = go.Figure()

for i,theta in enumerate([0.25,0.5,0.75,1,1.25,1.5]):
    ratios_mean = []
    times= []
    for j, file in enumerate(files[:103]):

        #fig.suptitle(f"{steps[j]} / {steps[-1]}")
        # top.cla()
        # bot.cla()

        omega = cell_corners_to_cell_center(torch.tensor(np.load(file)["omega"], dtype=torch.float64))[0,0,:,:]
        psi = torch.tensor(np.load(file)["p"], dtype=torch.float64) / 9.375e-5

        psi_top = psi[0,0,:,:]

        psi_bot = psi[0,1,:,:]

        above_thresh = torch.abs(psi_top) > theta*(torch.mean(torch.abs(psi_top)))
        
        ratios_mean.append(torch.mean(psi_bot[above_thresh] / psi_top[above_thresh]))

        # top.imshow(
        #     psi_top.T,
        #     cmap="bwr",
        #     vmin=-torch.max(torch.abs(psi_top)),
        #     vmax=torch.max(torch.abs(psi_top))
        # )
        # top.contour(~above_thresh.T, linestyles="-", colors='black')
        # top.contour(~not_null.T, linestyles="-", colors='grey')

        # bot.imshow(
        #     psi_bot.T,
        #     cmap="bwr",
        #     vmin=-torch.max(torch.abs(psi_bot)),
        #     vmax=torch.max(torch.abs(psi_bot))
        # )
        # bot.contour(~above_thresh.T, linestyles="-", colors='black')
        # bot.contour(~not_null.T, linestyles="-", colors='grey')

        times.append(steps[j] * run.configuration.simulation.dt)
    
    # np.savez(f"../data/coefficients_{str(theta).replace('.','_')}.npz", times = np.array(times), alpha=np.array(ratios_mean))
    
    fig.add_trace(
        go.Scatter(
            name = f"θ = {theta}",
            x=[t / 3600 / 24 for t in times],
            y=ratios_mean,
            mode = "lines",
            line= dict(width=8, color=colors[i]),
        )
    )

fig.update_layout(
    template="plotly_white",
    autosize=True,
    width=2500,
    height=1200 ,
    font={"size": 50, "color":"black"},
)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_xaxes(
    title={"text": "Time (day)"},
    exponentformat="e",
    mirror=True,
    linewidth=3,
    showgrid=False,
    linecolor="black",
)

fig.update_yaxes(
    title={"text": "α"},
    exponentformat="none",
    ticksuffix = "  ",
    tickprefix = "  ",
    tick0 = 0,
    nticks=5,
    mirror=True,
    showgrid=True,
    gridwidth=2,
    linewidth=3,
    linecolor="black",

)
fig.show()


0.2:

```python
    omega = cell_corners_to_cell_center(torch.tensor(np.load(file)["omega"], dtype=torch.float64))[0,0,24:-24,24:-24]
    psi = torch.tensor(np.load(file)["p"], dtype=torch.float64) / 9.375e-5

    psi_top = psi[0,0,24:-24,24:-24]
    psi_bot = psi[0,1,24:-24,24:-24]
    
    not_null = torch.abs(omega) > 1e-15

    above_thresh = torch.abs(psi_top) > (torch.mean(torch.abs(psi_top[not_null])))

    ratios_median.append(min(1,max(0,torch.median(psi_bot[above_thresh] / psi_top[above_thresh]))))
    ratios_mean.append(min(1,max(0,torch.mean(psi_bot[above_thresh] / psi_top[above_thresh]))))
```

0.4:

```python
    omega = cell_corners_to_cell_center(torch.tensor(np.load(file)["omega"], dtype=torch.float64))[0,0,3:-3,3:-3]
    psi = torch.tensor(np.load(file)["p"], dtype=torch.float64) / 9.375e-5

    psi_top = psi[0,0,3:-3,3:-3]
    psi_bot = psi[0,1,3:-3,3:-3]
    
    not_null = torch.abs(psi_top) > 1e-1

    above_thresh = torch.abs(psi_top) > (torch.mean(torch.abs(psi_top[not_null])))

    ratios_median.append(min(1,max(0,torch.median(psi_bot[above_thresh] / psi_top[above_thresh]))))
    ratios_mean.append(min(1,max(0,torch.mean(psi_bot[above_thresh] / psi_top[above_thresh]))))
```

In [ ]:
from qgsw.models.qg.alpha import ChangingCoefficient

coef = ChangingCoefficient.from_file("../data/coefficients_1_.npz")
coef.adjust_kernel_width(0.00744,9.375e-5)
coef.show()

In [ ]:
plt.scatter([i for i in range(30)],coef._generate_kernel())
plt.show()

In [ ]:
t_ = (torch.tensor(times) - 10*3600*24.0)
alpha_sigmoid = torch.sigmoid(t_/1e5)*0.2
print(alpha_sigmoid)
np.savez(f"../data/coefficients_sigmoid_0_2.npz", times = np.array(times), alpha=np.array(alpha_sigmoid))

In [ ]:
t_ = (torch.tensor(times) - 10*3600*24.0)
plt.plot(torch.sigmoid(t_/1e5)*0.25)
plt.show()

In [ ]:
plt.plot(alpha_sigmoid)
plt.show()

In [ ]:
psi_top_ = psi_top - torch.mean(psi_top)
plt.imshow(psi_top_, vmin=-torch.max(torch.abs(psi_top_)), vmax=torch.max(torch.abs(psi_top_)), cmap="bwr")
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(psi_bot)
plt.show()

In [ ]:
from qgsw.models.qg.alpha import ChangingCoefficient


coef = ChangingCoefficient.from_file("../data/coefficients_0_2.npz")

In [ ]:
coef.show()

In [ ]:
torch.rand((1,2,192,192))

# Metrics

In [ ]:
from datetime import datetime
import json

def parse_json(power_json: dict) -> dict:
    """Parse results Json.

    Args:
        power_json (dict): Power Json.

    Returns:
        dict: Parsed Json.
    """
    for data in power_json:
        time = data["timestamp"]
        data["timestamp"] = datetime.strptime(time, "%Y-%m-%dT%H:%M:%S.%f%z")
    return power_json

def read_g5000_api_file(file: Path) -> dict:
    """Read power consumption from a file.

    Args:
        file (Path): _description_

    Returns:
        dict: _description_
    """
    return parse_json(json.load(file.open("r")))

In [ ]:
folders = [
    "gpu_one_layer_model",
    "gpu_modified_model",
    "gpu_two_layers_model",
]

In [ ]:
from scipy.integrate import trapezoid

for folder in folders:

    durations = []
    energies = []
    powers = []

    folder_path = Path(f"../output/g5k/consumption/{folder}")
    files = folder_path.glob("bmc_node_power_watt_[0-9]*.json")
    for file in files:
        nb = file.stem[-7:]
        data = read_g5000_api_file(file)
        t0 = data[0]["timestamp"]
        times = [ (p["timestamp"] -t0 ).total_seconds() / 60 for p in data]
        power = [p["value"] for p in data]

        durations.append((data[-1]["timestamp"] - t0).total_seconds())

        energy = trapezoid(power,[t*60 for t in times])
        energies.append(energy)

        [powers.append(p) for p in power]

        plt.plot(times, power, label = f"{folder} - {nb}")
    print(folder, " : ", np.mean(durations) / 60, "min ± ", np.std(durations) / 60, "min")
    print(folder, " : ", np.mean(energies) / 1000, "kJ ± ", np.std(energies) / 1000, "kJ")
    print(folder, " : ", np.mean(powers), "W ± ", np.std(powers), "W")
    print("\n")

# plt.legend()
plt.show()
        

In [ ]:
mems = []
names = []
for folder in folders:

    folder_path = Path(f"../output/g5k/consumption/{folder}")
    
    jobs = [file.stem[-7:] for file in folder_path.glob("prom_node_memory_MemFree_bytes_[0-9]*.json")]


    buffers = []
    cached = []
    memfree = []
    memtot = []

    for job in jobs:

        data = read_g5000_api_file(folder_path.joinpath(f"prom_node_memory_Buffers_bytes_{job}.json"))
        buffers.append(np.array([p["value"] for p in data[4:]]))

        data = read_g5000_api_file(folder_path.joinpath(f"prom_node_memory_Cached_bytes_{job}.json"))
        cached.append(np.array([p["value"] for p in data[4:]]))
        
        data = read_g5000_api_file(folder_path.joinpath(f"prom_node_memory_MemFree_bytes_{job}.json"))
        memfree.append(np.array([p["value"] for p in data[4:]]))
        
        data = read_g5000_api_file(folder_path.joinpath(f"prom_node_memory_MemTotal_bytes_{job}.json"))
        memtot.append(np.array([p["value"] for p in data[4:]]))

    names.append([folder + " - " + job for job in jobs])

    mems.append([(memtot[i] - memfree[i] - cached[i] - buffers[i]) / memtot[i] * 100 for i in range(len(memtot))])

mem_1l = np.mean([np.mean(m_) for m_ in mems[0]])

print(mem_1l)

mem_alpha = np.mean([np.mean(m_) for m_ in mems[1]])

print(mem_alpha)

mem_2l = np.mean([np.mean(m_) for m_ in mems[2]])

print(mem_2l)

for i,mem in enumerate(mems):
    for j,m in enumerate(mem):

        print(names[i][j], ": ", np.mean(m) - np.mean([np.mean(m_) for m_ in mems[0]]))

    print(np.mean([np.mean(m) - np.mean([np.mean(m_) for m_ in mems[0]]) for m in mem]))
    